In [1]:
import matplotlib
%matplotlib tk

%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

# 

Autosaving every 180 seconds


In [2]:
##############################################
################# IMPORTS ####################
##############################################
import numpy as np
import os

#
import pandas as pd
import h5py
from tifffile import tifffile, imread
import pathlib
#import suite2p


###############################################
################ HELPER FUNCTIONS #############
###############################################

def get_mouse_ids(header,
                  dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='mouse ID')[0][0]

    ids = np.vstack(df.iloc[:,idx])

    return ids.squeeze()

def get_file_paths(header,
                  dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='path')[0][0]

    paths = np.vstack(df.iloc[:,idx])

    return paths.squeeze()

def get_sessions(header,
                 dfs):
    
    #
    header_ = np.array(header)

    idx = np.where(header_=='sessions')[0][0]

    sessions = np.vstack(df.iloc[:,idx][:])

    return sessions.squeeze()

def convert_mesc_sessions_to_concatenated_tiff(fname, 
                                               sessions_in):
    
    fname_out = fname.replace('.mesc','.tif')

    if os.path.exists(fname_out):
        return fname_out
    
    sessions_in = sessions_in.replace(" ", "").split (",")

    #
    sess_list = [] 
    for session in sessions_in:
        temp = session.replace("S",'')
        temp = 'MUnit_'+str(int(temp)-1)
        print ("session loaded: ", temp)
        sess_list.append(temp)
    #
    data = []
    with h5py.File(fname, 'r') as file:
                #
        for sess in sess_list:
            print ("processing: ", sess)
            temp = file['MSession_0'][sess]['Channel_0'][()]
            print ("    data loaded size: ", temp.shape)
            data.append(temp)

    data = np.vstack(data)
    print(data.shape)

    # from tifffile import tifffile
    
    tifffile.imwrite(fname_out, data)
    
    return fname_out


def run_suite2p_from_fname(fname):

    data = imread(fname)
    print('imaging data of shape: ', data.shape)
    n_time, Ly, Lx = data.shape

    #
    ops = suite2p.default_ops()
    ops['batch_size'] = 200 # we will decrease the batch_size in case low RAM on computer
    ops['threshold_scaling'] = 2.0 # we are increasing the threshold for finding ROIs to limit the number of non-cell ROIs found (sometimes useful in gcamp injections)
    ops['fs'] = 30 # sampling rate of recording, determines binning for cell detection
    ops['tau'] = 1.25 # timescale of gcamp to use for deconvolution
    ops['save_path0'] = os.path.split(fname)[0]
    ops['save_folder'] = os.path.split(fname)[0]
    ops['tiff_list'] = [fname]

    print(ops)

    #
    db = {
        'data_path': os.path.split(fname)[0],
    }
    print(db)

    #
    output_ops = suite2p.run_s2p(ops=ops, db=db)

      

In [20]:
#
fname = '/media/cat/4TB1/donato/anja/20221109.mesc'
sessions_in = 'S2'

#
data = []
with h5py.File(fname, 'r') as file:
            #
    temp = file['MSession_0']['MUnit_0']['Channel_0'][()]
    print (temp.shape)
    temp = file['MSession_0']['MUnit_1']['Channel_0']
    print (temp.shape)
    
            

(6, 512, 512)
(37161, 512, 512)


In [3]:
################################################################################
############################## READ SPREADSHEET ################################
################################################################################
#
fname = '/media/cat/4TB1/donato/DON-10775/22-08-10/data/Image_001_001.raw'
fname = '/media/cat/4TB/donato/andres/DON011733/20230118/calibration/Image_001_001.raw'
data = np.memmap(fname,
               dtype='uint16',
               mode='r')

data = data.reshape(-1, 512,512)
    
fname_out = fname.replace('.raw','.tiff')

tifffile.imwrite(fname_out, data)

#
print ("...DONE...")
    


...DONE...


In [12]:
import matplotlib.pyplot as plt

#
plt.figure()
plt.imshow(data[10000:11000].mean(0))
plt.colorbar()
plt.show()



In [31]:
from tqdm import tqdm


#image = cv2.imread('tif_file.tif')
#image = np.asarray(image,dtype = np.float64)

fnames= np.loadtxt('/home/cat/registered_tiff_list.txt', dtype='str')
#print (fnames)

# 
f = open('/media/cat/4TB1/donato/DON-10775/22-08-10/data/suite2p/plane0/data_registered.bin', 'ab')

#
ctr=0
for fname in tqdm(fnames):
    #print ("fname: ", fname)
    data = tifffile.imread(fname, key = np.arange(500))
    #data = cv2.imread(fname)
    #data = skio.imread(fname, plugin="tifffile")
    
    #print (data.shape)
    f.write(data)
    ctr+=1
    if ctr>20:
        break
        
#
f.close()


  0%|                                                                                                                 | 0/180 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TB1/donato/DON-10775/22-08-10/data/suite2p/plane0/reg_tif/file000_chan0.tif'

In [30]:
fname = '/media/cat/4TB1/donato/DON-10775/22-08-10/bmi_rerun/data/Image_001_001.raw'
data = np.memmap(fname,
               dtype='uint16',
               mode='r')

data = data.reshape(-1, 512,512)
print (data.shape)

ValueError: cannot reshape array of size 25248792576 into shape (512,512)

In [32]:
import suite2p, numpy as np

def load_params(fname_ops, ops):
    
    ps_read = np.load(fname_ops,
                  allow_pickle=True).tolist()
    #print ("ops_read: ", ops_read)
    print (len(ops_read))
    names = []
    vals = []
    for o in ops_read:
        ops[o] = ops_read[o]    
        
    return ops
#
fname_ops = '/media/cat/4TB1/donato/steffen/scicore/Steffen/ops_CA3.npy'
ops = suite2p.default_ops()

ops = load_params(fname_ops, ops)
print (ops)
#ops['batch_size'] = 200 # we will decrease the batch_size in case low RAM on computer
#ops['threshold_scaling'] = 2.0 # we are increasing the threshold for finding ROIs to limit the number of non-cell ROIs found (sometimes useful in gcamp injections)
#ops['fs'] = 30 # sampling rate of recording, determines binning for cell detection
#ops['tau'] = 1.25 # timescale of gcamp to use for deconvolution
#ops['save_path0'] = os.path.split(fname)[0]
#ops['save_folder'] = os.path.split(fname)[0]
#ops['tiff_list'] = [fname]

#print(ops)



86
{'suite2p_version': '0.10.1', 'look_one_level_down': 0.0, 'fast_disk': [], 'delete_bin': True, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'nwb_file': '', 'nwb_driver': '', 'nwb_series': '', 'save_path0': [], 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 30.947, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': 0.0, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': True, 'save_NWB': 0.0, 'combined': 1.0, 'aspect': 1.0, 'do_bidiphase': False, 'bidiphase': 0.0, 'bidi_corrected': False, 'do_registration': 1, 'two_step_registration': 0.0, 'keep_movie_raw': False, 'nimg_init': 500, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': True, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0.0, 'smooth_sigma': 1.15, 'th_badframes': 100.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'nonrig